In [1]:
import os
import pandas as pd

In [2]:
# --- 1. Paths & filenames ---
DATA_DIR = r'/home/muhammad/Education/Data Mining Lab/Project/mining/json/data'
WEATHER_FILES = [
    'dallas.json','houston.json','la.json','nyc.json',
    'san_diego.json','san_jose.json','san_antonio.json',
    'phoenix.json','philadelphia.json','seattle.json'
]
DEMAND_FILES = {
    'nyc': 'cleaned_subregion_data.csv',
    'phoenix': 'cleaned_balance_data.csv',
    'seattle': 'cleaned_balance_data.csv',
    'houston': 'cleaned_texas_data.csv',
    'san antonio': 'cleaned_texas_data.csv',
    'dallas': 'cleaned_texas_data.csv'
}

In [3]:
# --- 2. Load & unify weather JSONs ---
weather_dfs = []
for fname in WEATHER_FILES:
    city_key = os.path.splitext(fname)[0].replace('_', ' ').lower()
    file_path = os.path.join(DATA_DIR, fname)
    df = pd.read_json(file_path)
    df['city'] = city_key
    df['timestamp'] = pd.to_datetime(df['time'], unit='s')
    df.drop(columns=['time'], inplace=True)
    weather_dfs.append(df)
weather_df = pd.concat(weather_dfs, ignore_index=True)
weather_df.to_csv(os.path.join(DATA_DIR, 'weather_data.csv'), index=False)

In [4]:
# --- 3. Load & unify demand sources ---
demand_dfs = []
for city, fname in DEMAND_FILES.items():
    path = os.path.join(DATA_DIR, fname)
    df = pd.read_csv(path)
    # parse timestamp column
    if 'local_time' in df.columns:
        df['timestamp'] = pd.to_datetime(df['local_time'])
    elif 'date' in df.columns:
        df['timestamp'] = pd.to_datetime(df['date'])
    # select & rename demand
    df = df.rename(columns={'demand': 'demand_mwh'})
    # ensure city column exists
    if 'city' not in df.columns:
        # texas file: pivot wide to long
        df = df.melt(
            id_vars=['timestamp'],
            value_vars=[c for c in df.columns if c not in ['timestamp','date']],
            var_name='city', value_name='demand_mwh'
        )
        df['city'] = df['city'].str.lower()
    else:
        df['city'] = df['city'].str.lower()
    demand_dfs.append(df[['timestamp','city','demand_mwh']])
demand_df = pd.concat(demand_dfs, ignore_index=True)
demand_df.to_csv(os.path.join(DATA_DIR, 'demand_data.csv'), index=False)

In [5]:
# --- 4. Merge weather & demand ---
combined_df = pd.merge(
    weather_df,
    demand_df,
    on=['timestamp','city'],
    how='left'
)
combined_df.to_csv(os.path.join(DATA_DIR, 'combined_data.csv'), index=False)